Generating code for a model:

In [1]:
import pytorch_composer

input_dim = [4,3,32,32]
# A random sequence of neural network layers. Any positive integer shoud be a valid dimension arguement:
sequence = [
    ["conv2d", 6],
    ["maxpool2d", 2],
    ["linear", 52],
    ["relu"],
    ["maxpool2d", 2],
    ["linear",43],
    ["maxpool2d", 2],
    ["conv2d", 65],
    ["maxpool2d", 2],
    ["conv2d", 47],
    ["flat"],
    ["linear", 52],
    ["relu"],
    ["linear",38],
]

model_code = pytorch_composer.write_model(input_dim, sequence)
model_code.print_formated()

class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.conv2d1 = nn.Conv2d(3, 6, kernel_size = 3, padding = 0)
		self.maxpool2d1 = nn.MaxPool2d(kernel_size = 2, padding = 0)
		self.linear1 = nn.Linear(15, 52)
		self.linear2 = nn.Linear(26, 43)
		self.conv2d2 = nn.Conv2d(6, 65, kernel_size = 3, padding = 0)
		self.maxpool2d2 = nn.MaxPool2d(kernel_size = 2, padding = (1, 0))
		self.conv2d3 = nn.Conv2d(65, 47, kernel_size = 3, padding = (1, 0))
		self.linear3 = nn.Linear(329, 52)
		self.linear4 = nn.Linear(52, 38)
	def forward(self, x):
		# Input dimensions : [4, 3, 32, 32]
		# Shape goes from [4, 3, 32, 32] to [4, 6, 30, 30]
		x = self.conv2d1(x)
		# Shape goes from [4, 6, 30, 30] to [4, 6, 15, 15]
		x = self.maxpool2d1(x)
		# Shape goes from [4, 6, 15, 15] to [4, 6, 15, 52]
		x = self.linear1(x)
		x = F.relu(x)
		# Shape goes from [4, 6, 15, 52] to [4, 6, 7, 26]
		x = self.maxpool2d0(x)
		# Shape goes from [4, 6, 7, 26] to [4, 6, 7, 43]
		x = self.linear2(x)
		#

Using the generated code:

In [2]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Now we run the generated code:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, kernel_size = 3, padding = 0)
        self.maxpool2d1 = nn.MaxPool2d(kernel_size = 2, padding = 0)
        self.linear1 = nn.Linear(1350, 52)
        self.linear2 = nn.Linear(52, 43)
        self.maxpool2d2 = nn.MaxPool2d(kernel_size = 2, padding = (0, 1))
        self.conv2d2 = nn.Conv2d(1, 65, kernel_size = 3, padding = (0, 1))
        self.conv2d3 = nn.Conv2d(65, 47, kernel_size = 3, padding = (0, 1))
        self.linear3 = nn.Linear(329, 52)
        self.linear4 = nn.Linear(52, 38)
    def forward(self, x):
        # Input dimensions : [4, 3, 32, 32]
        # Shape goes from [4, 3, 32, 32] to [4, 6, 30, 30]
        x = self.conv2d1(x)
        # Shape goes from [4, 6, 30, 30] to [4, 6, 15, 15]
        x = self.maxpool2d1(x)
        # Reshaping the data from [4, 6, 15, 15] to [4, 1350]:
        x = x.view(-1,1350)
        # Shape goes from [4, 1350] to [4, 52]
        x = self.linear1(x)
        x = F.relu(x)
        # Shape goes from [4, 52] to [4, 43]
        x = self.linear2(x)
        # Reshaping the data from [4, 43] to [4, 1, 43, 1]:
        x = x.view(4, 1, 43, 1)
        # Shape goes from [4, 1, 43, 1] to [4, 1, 21, 1]
        x = self.maxpool2d2(x)
        # Shape goes from [4, 1, 21, 1] to [4, 65, 19, 1]
        x = self.conv2d2(x)
        # Shape goes from [4, 65, 19, 1] to [4, 65, 9, 1]
        x = self.maxpool2d1(x)
        # Shape goes from [4, 65, 9, 1] to [4, 47, 7, 1]
        x = self.conv2d3(x)
        # Reshaping the data from [4, 47, 7, 1] to [4, 329]:
        x = x.view(-1,329)
        # Shape goes from [4, 329] to [4, 52]
        x = self.linear3(x)
        x = F.relu(x)
        # Shape goes from [4, 52] to [4, 38]
        x = self.linear4(x)
        return x

In [4]:
# Define a Loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Training
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        torch.save(net.state_dict(), 'model.pt')

print('Finished Training')

[1,  2000] loss: 2.498
[1,  4000] loss: 2.260
[1,  6000] loss: 1.928
[1,  8000] loss: 1.771
[1, 10000] loss: 1.655
[1, 12000] loss: 1.558
[2,  2000] loss: 1.449
[2,  4000] loss: 1.429
[2,  6000] loss: 1.418
[2,  8000] loss: 1.376
[2, 10000] loss: 1.341
[2, 12000] loss: 1.316
Finished Training
